In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
xl = pd.ExcelFile("./raw_data/NWM_data_updated.xlsx", engine='openpyxl')
a=xl.sheet_names

#loading the data set
dataframe_collection = {}
i=0
for i in range(15):
    if i==0 :
        dataframe_collection[i] = xl.parse(a[i],converters={"STAID_1":str})
    else:
        dataframe_collection[i] = xl.parse(a[i])


obsflow = dataframe_collection[2]
modelflow= dataframe_collection[13]
station_data_total = dataframe_collection[0]

In [6]:
split = 0.6
for i in range(0,7):
    split=split+0.05
    for v in range(0,5):  
                print(v)
                station_data = station_data_total.sample(frac=split)
                station_data_ungagued = station_data_total.loc[~station_data_total.index.isin(station_data.index)]

                station_data = station_data.reset_index(drop=True)
                station_data_ungagued = station_data_ungagued.reset_index(drop=True)



                k=0
                column_names = ["dis2sea","landUse","soilTexture1", "soilTexture2","soilTexture3","soilTexture4","soilTexture5","soilTexture6","soilTexture7","soilTexture8","soilTexture9","soilTexture10","topoIndex","drainageArea","ModelFlow","Obsflow"]
                df = pd.DataFrame(columns = column_names)  
                for index, row in station_data.iterrows():
                    print(index)
                    sta_id = station_data.iloc[index,0]
                    dis2sea=station_data.iloc[index,1]
                    land_use=station_data.iloc[index,2]
                    soil_texture1=station_data.iloc[index,3]
                    soil_texture2=station_data.iloc[index,4]
                    soil_texture3=station_data.iloc[index,5]
                    soil_texture4=station_data.iloc[index,6]
                    soil_texture5=station_data.iloc[index,7]
                    soil_texture6=station_data.iloc[index,8]
                    soil_texture7=station_data.iloc[index,9]
                    soil_texture8=station_data.iloc[index,10]
                    soil_texture9=station_data.iloc[index,11]
                    soil_texture10=station_data.iloc[index,12]
                    topoindex = station_data.iloc[index,13]
                    drainagearea =  station_data.iloc[index,14]
                    obs = obsflow[sta_id]/drainagearea
                    mod = modelflow[sta_id]/drainagearea

                    for i in range(979):
                        df.loc[k]=[dis2sea]+[land_use]+[soil_texture1]+[soil_texture2]+[soil_texture3]+[soil_texture4]+[soil_texture5]+[soil_texture6]+[soil_texture7]+[soil_texture8]+[soil_texture9]+[soil_texture10]+[topoindex]+[drainagearea]+[mod[i]]+[obs[i]]
                        k=k+1
                df =df.dropna()



                #train_test Split

                import numpy as np
                from sklearn.model_selection import train_test_split
                df=df.sample(frac=1).reset_index(drop=True)
                df.iloc[:,15]= abs(df.iloc[:,15] - df.iloc[:,14])

                x, y = df.iloc[:,0:15], df.iloc[:,15]
                y = y.values.reshape(y.shape[0],1)
                x_train, x_test, y_train, y_test = \
                    train_test_split(x, y, test_size=0.1, 
                                                 random_state=0)

                #standardize
                x_mean = x_train.mean(axis=0)
                x_train -= x_mean
                x_std = x_train.std(axis=0)
                x_train /= x_std

                x_test -= x_mean
                x_test /= x_std 

                y_mean = y_train.mean(axis=0)
                y_train -=y_mean
                y_std=y_train.std(axis=0)
                y_train /=y_std

                y_test -=y_mean
                y_test /=y_std


                #model Train
                from keras.models import Sequential
                from keras.layers import Dense
                from keras import layers
                from keras.layers import LSTM
                from keras import backend as K 


                model= Sequential()

                model.add(layers.Dense(512, activation='relu',input_shape=(x_train.shape[1],)))
                model.add(layers.Dense(256, activation='relu'))
                model.add(layers.Dense(64, activation='relu'))
                model.add(layers.Dense(32, activation='relu'))
                model.add(layers.Dense(16, activation='relu'))
               # model.add(layers.Drut(opo0.4))
    #model.add(layers.Dense(4, activation='relu'))
                model.add(layers.Dense(1))
                model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
                #end edit    
                # Fitting the RNN to the Training set
                history=model.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size = 1024, epochs = 15)
                #istory=model.fit(x_train, y_train, batch_size = 1000, epochs = 20) 3
                model.save('/models/my_model_10_day_bestv2.h5')

                mae = pd.DataFrame(history.history['mae'])
                val_mae= pd.DataFrame(history.history['val_mae'])

                d_mae=pd.concat([mae, val_mae], axis=1)
                d_mae.columns = ['mae', 'val_mae']
                d_mae

                #d_mae.to_csv('mae_iter_%s_bestv2.csv'%v)



                #Preparing Second dataset


                k=0
                column_names = ["dis2sea","landUse","soilTexture1", "soilTexture2","soilTexture3","soilTexture4","soilTexture5","soilTexture6","soilTexture7","soilTexture8","soilTexture9","soilTexture10","topoIndex","drainageArea","ModelFlow","Obsflow"]
                df2 = pd.DataFrame(columns = column_names)  
                for index, row in station_data_ungagued.iterrows():
                    print(index)
                    sta_id = station_data_ungagued.iloc[index,0]
                    dis2sea=station_data_ungagued.iloc[index,1]
                    land_use=station_data_ungagued.iloc[index,2]
                    soil_texture1=station_data_ungagued.iloc[index,3]
                    soil_texture2=station_data_ungagued.iloc[index,4]
                    soil_texture3=station_data_ungagued.iloc[index,5]
                    soil_texture4=station_data_ungagued.iloc[index,6]
                    soil_texture5=station_data_ungagued.iloc[index,7]
                    soil_texture6=station_data_ungagued.iloc[index,8]
                    soil_texture7=station_data_ungagued.iloc[index,9]
                    soil_texture8=station_data_ungagued.iloc[index,10]
                    soil_texture9=station_data_ungagued.iloc[index,11]
                    soil_texture10=station_data_ungagued.iloc[index,12]
                    topoindex = station_data_ungagued.iloc[index,13]
                    drainagearea =  station_data_ungagued.iloc[index,14]
                    obs = obsflow[sta_id]/drainagearea
                    mod = modelflow[sta_id]/drainagearea

                    for i in range(979):
                        df2.loc[k]=[dis2sea]+[land_use]+[soil_texture1]+[soil_texture2]+[soil_texture3]+[soil_texture4]+[soil_texture5]+[soil_texture6]+[soil_texture7]+[soil_texture8]+[soil_texture9]+[soil_texture10]+[topoindex]+[drainagearea]+[mod[i]]+[obs[i]]
                        k=k+1
                df2 =df2.dropna()

                x, y = df2.iloc[:, 0:15], df2.iloc[:,15]
                x-= x_mean
                x /= x_std



                from keras.models import load_model
                model = load_model('spatial_DLmodels/my_model_10_day.h5')
                model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

                z=model.predict(x)

                x= x*x_std
                x=x+x_mean
                d_area_x = x.iloc[:,13]
                x.iloc[:,14]=x.iloc[:,14]*d_area_x

                y=y*d_area_x

                z=z*y_std
                z=z+y_mean
                d_area_x=np.array(d_area_x)
                d_area_x.shape = [d_area_x.shape[0],1]
                z=z*d_area_x

                d_x= pd.DataFrame(x.iloc[:,14])
                d_y=pd.DataFrame(y)
                d_z=pd.DataFrame(z)

                d_xy=pd.concat([d_x, d_y,d_z], axis=1)
                d_xy.columns = ['Modelflow', 'Obsflow','Error']
                d_xy

                d_xy.to_csv("./graph_data/temp/Error_WholeData_abs_inter_bestv2_%s_%s.csv"%(split,v))




0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
Epoch 1/15
154/154 [==============================] - 2s 8ms/step - loss: 0.7354 - mae: 0.1551 - val_loss: 0.8466 - val_mae: 0.1379
Epoch 2/15
154/154 [==============================] - 1s 7ms/step - loss: 0.7099 - mae: 0.1439 - val_loss: 0.8325 - val_mae: 0.1646


Epoch 11/15
155/155 [==============================] - 1s 7ms/step - loss: 0.6897 - mae: 0.1367 - val_loss: 0.7266 - val_mae: 0.1473
Epoch 12/15
155/155 [==============================] - 1s 6ms/step - loss: 0.6934 - mae: 0.1371 - val_loss: 0.6943 - val_mae: 0.1380
Epoch 13/15
155/155 [==============================] - 1s 7ms/step - loss: 0.6926 - mae: 0.1360 - val_loss: 0.6982 - val_mae: 0.1624
Epoch 14/15
155/155 [==============================] - 1s 7ms/step - loss: 0.6908 - mae: 0.1357 - val_loss: 0.7914 - val_mae: 0.1577
Epoch 15/15
155/155 [==============================] - 1s 7ms/step - loss: 0.6942 - mae: 0.1368 - val_loss: 0.7002 - val_mae: 0.1426
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
1

Epoch 3/15
167/167 [==============================] - 2s 9ms/step - loss: 0.6950 - mae: 0.1591 - val_loss: 0.7125 - val_mae: 0.1551
Epoch 4/15
167/167 [==============================] - 2s 9ms/step - loss: 0.6874 - mae: 0.1572 - val_loss: 0.6872 - val_mae: 0.1916
Epoch 5/15
167/167 [==============================] - 2s 9ms/step - loss: 0.6850 - mae: 0.1569 - val_loss: 0.7025 - val_mae: 0.1510
Epoch 6/15
167/167 [==============================] - 2s 9ms/step - loss: 0.6884 - mae: 0.1582 - val_loss: 0.6812 - val_mae: 0.1553
Epoch 7/15
167/167 [==============================] - 2s 9ms/step - loss: 0.6838 - mae: 0.1580 - val_loss: 0.6949 - val_mae: 0.1587
Epoch 8/15
167/167 [==============================] - 2s 9ms/step - loss: 0.6821 - mae: 0.1578 - val_loss: 0.6686 - val_mae: 0.1453
Epoch 9/15
167/167 [==============================] - 1s 9ms/step - loss: 0.6831 - mae: 0.1585 - val_loss: 0.6755 - val_mae: 0.1510
Epoch 10/15
167/167 [==============================] - 1s 9ms/step - loss: 0

19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
Epoch 1/15
167/167 [==============================] - 2s 10ms/step - loss: 0.7446 - mae: 0.1657 - val_loss: 0.7083 - val_mae: 0.1407
Epoch 2/15
167/167 [==============================] - 2s 10ms/step - loss: 0.7206 - mae: 0.1554 - val_loss: 0.70

Epoch 2/15
179/179 [==============================] - 1s 8ms/step - loss: 0.6904 - mae: 0.1639 - val_loss: 0.5205 - val_mae: 0.1504
Epoch 3/15
179/179 [==============================] - 1s 8ms/step - loss: 0.6860 - mae: 0.1612 - val_loss: 0.5213 - val_mae: 0.1444
Epoch 4/15
179/179 [==============================] - 1s 8ms/step - loss: 0.6827 - mae: 0.1601 - val_loss: 0.5163 - val_mae: 0.1416
Epoch 5/15
179/179 [==============================] - 1s 8ms/step - loss: 0.6822 - mae: 0.1581 - val_loss: 0.5252 - val_mae: 0.1727
Epoch 6/15
179/179 [==============================] - 1s 8ms/step - loss: 0.6793 - mae: 0.1584 - val_loss: 0.5122 - val_mae: 0.1633
Epoch 7/15
179/179 [==============================] - 1s 8ms/step - loss: 0.6786 - mae: 0.1580 - val_loss: 0.5394 - val_mae: 0.1500
Epoch 8/15
179/179 [==============================] - 1s 8ms/step - loss: 0.6797 - mae: 0.1583 - val_loss: 0.5217 - val_mae: 0.1614
Epoch 9/15
179/179 [==============================] - 1s 8ms/step - loss: 0.

...

211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
Epoch 1/15
203/203 [==============================] - 2s 8ms/step - loss: 0.7384 - mae: 0.1531 - val_loss: 0.5664 - val_mae: 0.1402
Epoch 2/15
203/203 [==============================] - 1s 7ms/step - loss: 0.7182 - mae: 0.1431 - val_loss: 0.5002 - val_mae: 0.1305
Epoch 3/15
203/203 [==============================] - 1s 7ms/step - loss: 0.7062 - mae: 0.1404 - val_loss: 0.5182 - val_mae: 0.1355
Epoch 4/15
203/203 [==============================] - 1s 7ms/step - loss: 0.7083 - mae: 0.1398 - val_loss: 0.4935 - val_mae: 0.1388
Epoch 5/15
203/203 [==============================] - 1s 7ms/step - loss: 0.7067 - mae: 0.1400 - val_loss: 0.4807 - val_mae: 0.1353
Epoch 6/15
203/203 [==============================] - 1s 7ms/step - loss: 0.7046

Epoch 14/15
203/203 [==============================] - 1s 7ms/step - loss: 0.6862 - mae: 0.1517 - val_loss: 0.7862 - val_mae: 0.1599
Epoch 15/15
203/203 [==============================] - 1s 7ms/step - loss: 0.6861 - mae: 0.1521 - val_loss: 0.7716 - val_mae: 0.1591
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
1

88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
Epoch 1/15
214/214 [==============================] - 2s 7ms/step - loss: 0.7295 - mae: 0.1672 - val_loss: 0.5630 - val_mae: 0.1517
Epoch 2/15
214/214 [==============================] - 1s 7ms

Epoch 9/15
226/226 [==============================] - 1s 6ms/step - loss: 0.6901 - mae: 0.1477 - val_loss: 0.6548 - val_mae: 0.1432
Epoch 10/15
226/226 [==============================] - 1s 6ms/step - loss: 0.6902 - mae: 0.1469 - val_loss: 0.6566 - val_mae: 0.1510
Epoch 11/15
226/226 [==============================] - 1s 6ms/step - loss: 0.6892 - mae: 0.1480 - val_loss: 0.6589 - val_mae: 0.1534
Epoch 12/15
226/226 [==============================] - 1s 6ms/step - loss: 0.6874 - mae: 0.1478 - val_loss: 0.6577 - val_mae: 0.1651
Epoch 13/15
226/226 [==============================] - 1s 6ms/step - loss: 0.6864 - mae: 0.1477 - val_loss: 0.6522 - val_mae: 0.1532
Epoch 14/15
226/226 [==============================] - 1s 6ms/step - loss: 0.6863 - mae: 0.1471 - val_loss: 0.6497 - val_mae: 0.1752
Epoch 15/15
226/226 [==============================] - 1s 6ms/step - loss: 0.6873 - mae: 0.1469 - val_loss: 0.6459 - val_mae: 0.1445
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
2
0
1
2
3
4
5
6
7
8
9
10
11
12
1

258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
Epoch 1/15
228/228 [==============================] - 3s 9ms/step - loss: 0.7371 - mae: 0.1637 - val_loss: 0.6636 - val_mae: 0.1462
Epoch 2/15
228/228 [==============================] - 2s 9ms/step - loss: 0.7118 - mae: 0.1548 - val_loss: 0.6376 - val_mae: 0.1501
Epoch 3/15
228/228 [==============================] - 2s 9ms/step - loss: 0.7050 - mae: 0.1527 - val_loss: 0.6624 - val_mae: 0.1370
Epoch 4/15
228/228 [==============================] - 2s 9ms/step - loss: 0.7011 - mae: 0.1516 - val_loss: 0.6325 - val_mae: 0.1547
Epoch 5/15
228/228 [==============================] - 2s 9ms/step - loss: 0.6956 - mae: 0.1504 - val_loss: 0.6427 - val_mae: 0.1710
Epoch 6/15
228/228 [==============================] - 2s 9ms/step - loss: 0.6985 - mae: 0.1509 - val_loss: 0.6229 - val_mae: 0.1450
Epoc